## Importing the Packages

In [1]:
import numpy as np

In [2]:
np.set_printoptions(suppress=True, linewidth=200, precision=2)

## Importing the Data

In [3]:
rawData = np.genfromtxt("loan-data.csv", delimiter=';', skip_header= 1, autostrip=True, )
print(rawData)

[[48010226.           nan    35000.   ...         nan         nan     9452.96]
 [57693261.           nan    30000.   ...         nan         nan     4679.7 ]
 [59432726.           nan    15000.   ...         nan         nan     1969.83]
 ...
 [50415990.           nan    10000.   ...         nan         nan     2185.64]
 [46154151.           nan         nan ...         nan         nan     3199.4 ]
 [66055249.           nan    10000.   ...         nan         nan      301.9 ]]


## Checking for Incomplete Data

In [4]:
print(np.isnan(rawData).sum())

88005


In [5]:
temporaryFill = np.nanmax(rawData) + 1
temporaryMean = np.nanmean(rawData, axis=0)

/tmp/ipykernel_6522/3254766337.py:2: RuntimeWarning: Mean of empty slice
  temporaryMean = np.nanmean(rawData, axis=0)


In [6]:
temporaryFill

np.float64(68616520.0)

In [7]:
temporaryMean


array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,      440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [8]:
temporaryStats = np.array([np.nanmin(rawData,axis=0),
                           temporaryMean,
                           np.nanmax(rawData,axis=0)
                           ])

/tmp/ipykernel_6522/3501726121.py:1: RuntimeWarning: All-NaN slice encountered
  temporaryStats = np.array([np.nanmin(rawData,axis=0),
/tmp/ipykernel_6522/3501726121.py:3: RuntimeWarning: All-NaN slice encountered
  np.nanmax(rawData,axis=0)


In [9]:
temporaryStats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,       31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,      440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,     1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

## Splitting the Dataset

### Splitting the Columns

In [10]:
columnStrings = np.argwhere(np.isnan(temporaryMean)).squeeze()
columnStrings


array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [11]:
columnsNumeric = np.argwhere(np.isnan(temporaryMean) == False).squeeze()
columnsNumeric

array([ 0,  2,  4,  6,  7, 13])

### Re-importing the Dataset

In [12]:
rawDataStrings = np.genfromtxt("loan-data.csv", delimiter=';', skip_header= 1, autostrip=True, usecols=columnStrings, dtype=np.str_)
rawDataStrings

array([['May-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']], shape=(10000, 8), dtype='<U69')

In [13]:
rawDataNumbers = np.genfromtxt('loan-data.csv', delimiter=';', skip_header=1,autostrip=True, usecols=columnsNumeric, filling_values= temporaryFill)
rawDataNumbers

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]], shape=(10000, 6))

### The Names of the Columns

In [14]:
headerFull = np.genfromtxt("loan-data.csv", delimiter=';', skip_footer= rawData.shape[0], autostrip=True,  dtype=np.str_)
headerFull

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state', 'total_pymnt'], dtype='<U19')

In [15]:
headerStrings = headerFull[columnStrings]
headerStrings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state'], dtype='<U19')

In [16]:
headerNumerics = headerFull[columnsNumeric]
headerNumerics

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## Creating Checkpoints:

In [17]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

In [18]:
checkpointAfterTerm = checkpoint('checkpoint-AfterTerm', headerStrings,rawDataStrings)
checkpointAfterTerm['data']

array([['May-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']], shape=(10000, 8), dtype='<U69')

## Manipulating String Columns

In [19]:
headerStrings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state'], dtype='<U19')

In [20]:
headerStrings[0] = 'issueDate'

In [21]:
rawDataStrings

array([['May-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']], shape=(10000, 8), dtype='<U69')

### Issue Date

In [22]:
np.unique(rawDataStrings[:, 0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15', 'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [23]:
rawDataStrings[:,0] = np.chararray.strip(rawDataStrings[:,0], '-15')
np.unique(rawDataStrings[:,0])

/tmp/ipykernel_6522/1112911887.py:1: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  rawDataStrings[:,0] = np.chararray.strip(rawDataStrings[:,0], '-15')


array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'], dtype='<U69')

In [24]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [25]:
for i in range(13):
        rawDataStrings[:,0] = np.where(rawDataStrings[:,0] == str(months[i]),
                                          str(i),
                                          rawDataStrings[:,0])

In [26]:
np.unique(rawDataStrings[:, 0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Loan Status

In [27]:
headerStrings

array(['issueDate', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state'], dtype='<U19')

In [28]:
rawDataStrings[:,1]

array(['Current', 'Current', 'Current', ..., 'Current', 'Current', 'Current'], shape=(10000,), dtype='<U69')

In [29]:
np.unique(rawDataStrings[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued', 'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [30]:
statusBad = np.array(['', 'Charged Off','Default', 'Late (31-120 days)'])

In [31]:
rawDataStrings[:,1] = np.where(np.isin(rawDataStrings[:,1],statusBad),0,1)

In [32]:
np.unique(rawDataStrings[:,1])

array(['0', '1'], dtype='<U69')

### Term

In [33]:
headerStrings

array(['issueDate', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state'], dtype='<U19')

In [34]:
rawDataStrings[:,2]

array(['36 months', '36 months', '36 months', ..., '36 months', '36 months', '36 months'], shape=(10000,), dtype='<U69')

In [35]:
np.unique(rawDataStrings[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

In [36]:
rawDataStrings[:,2] = np.chararray.strip(rawDataStrings[:,2],' months')

/tmp/ipykernel_6522/1819007729.py:1: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  rawDataStrings[:,2] = np.chararray.strip(rawDataStrings[:,2],' months')


In [37]:
np.unique(rawDataStrings[:,2])

array(['', '36', '60'], dtype='<U69')

In [38]:
headerStrings[2] = 'termMonths'

In [39]:
rawDataStrings[:,2] = np.where(rawDataStrings[:,2] == '',
                               '60',
                               rawDataStrings[:,2])

In [40]:
np.unique(rawDataStrings[:,2])


array(['36', '60'], dtype='<U69')

### Grade and Subgrade

In [41]:
headerStrings

array(['issueDate', 'loan_status', 'termMonths', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state'], dtype='<U19')

In [42]:
rawDataStrings[:,3]
np.unique(rawDataStrings[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

#### Filling Sub Grade

In [43]:
for i in np.unique(rawDataStrings[:,3])[1:]:
    rawDataStrings[:,4] = np.where((rawDataStrings[:,4] == '') & (rawDataStrings[:,3] == i),
                                      i + '5',
                                      rawDataStrings[:,4])

In [44]:
np.unique(rawDataStrings[:,4], return_counts=True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1',
        'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267, 250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5]))

In [45]:
rawDataStrings[:,4] = np.where(rawDataStrings[:,4] == '',
                               'H1',
                               rawDataStrings[:,4])

In [46]:
np.unique(rawDataStrings[:,4], return_counts=True)

(array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2',
        'G3', 'G4', 'G5', 'H1'], dtype='<U69'),
 array([285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267, 250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5,   9]))

#### Removing Grade

In [47]:
rawDataStrings = np.delete(rawDataStrings,3,axis= 1)

In [48]:
rawDataStrings[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], shape=(10000,), dtype='<U69')

In [49]:
headerStrings = np.delete(headerStrings,3)
headerStrings

array(['issueDate', 'loan_status', 'termMonths', 'sub_grade', 'verification_status', 'url', 'addr_state'], dtype='<U19')

In [50]:
headerStrings[3]

np.str_('sub_grade')

#### Converting Sub Grade

In [51]:
np.unique(rawDataStrings[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2',
       'G3', 'G4', 'G5', 'H1'], dtype='<U69')

In [52]:
keys = list(np.unique(rawDataStrings[:,3]))                         
values = list(range(1, np.unique(rawDataStrings[:,3]).shape[0] + 1)) 
dict_sub_grade = dict(zip(keys, values))

In [53]:
dict_sub_grade

{np.str_('A1'): 1,
 np.str_('A2'): 2,
 np.str_('A3'): 3,
 np.str_('A4'): 4,
 np.str_('A5'): 5,
 np.str_('B1'): 6,
 np.str_('B2'): 7,
 np.str_('B3'): 8,
 np.str_('B4'): 9,
 np.str_('B5'): 10,
 np.str_('C1'): 11,
 np.str_('C2'): 12,
 np.str_('C3'): 13,
 np.str_('C4'): 14,
 np.str_('C5'): 15,
 np.str_('D1'): 16,
 np.str_('D2'): 17,
 np.str_('D3'): 18,
 np.str_('D4'): 19,
 np.str_('D5'): 20,
 np.str_('E1'): 21,
 np.str_('E2'): 22,
 np.str_('E3'): 23,
 np.str_('E4'): 24,
 np.str_('E5'): 25,
 np.str_('F1'): 26,
 np.str_('F2'): 27,
 np.str_('F3'): 28,
 np.str_('F4'): 29,
 np.str_('F5'): 30,
 np.str_('G1'): 31,
 np.str_('G2'): 32,
 np.str_('G3'): 33,
 np.str_('G4'): 34,
 np.str_('G5'): 35,
 np.str_('H1'): 36}

In [54]:
for i in np.unique(rawDataStrings[:,3]):
        rawDataStrings[:,3] = np.where(rawDataStrings[:,3] == i, 
                                          str(dict_sub_grade[i]),
                                          rawDataStrings[:,3])

In [55]:
np.unique(rawDataStrings[:,3], return_counts=True)

(array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '4', '5', '6',
        '7', '8', '9'], dtype='<U69'),
 array([285, 633, 629, 567, 586, 564, 577, 391, 267, 250, 255, 278, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43, 239,  24,  19,  10,   3,   7,   5,   9, 323, 592, 509, 517, 530, 553]))

### Verification Status

In [56]:
headerStrings

array(['issueDate', 'loan_status', 'termMonths', 'sub_grade', 'verification_status', 'url', 'addr_state'], dtype='<U19')

In [57]:
np.unique(rawDataStrings[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

In [58]:
rawDataStrings[:,4] = np.where((rawDataStrings[:,4]=='') | (rawDataStrings[:,4] == 'Not Verified'), 0, 1)

In [59]:
np.unique(rawDataStrings[:,4])

array(['0', '1'], dtype='<U69')

### URL

In [60]:
rawDataStrings[:,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ..., 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], shape=(10000,), dtype='<U69')

In [61]:
rawDataStrings[:,5] = np.chararray.strip(rawDataStrings[:,5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')

/tmp/ipykernel_6522/503279534.py:1: DeprecationWarning: `np.chararray` is deprecated and will be removed from the main namespace in the future. Use an array with a string or bytes dtype instead.
  rawDataStrings[:,5] = np.chararray.strip(rawDataStrings[:,5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')


In [62]:
rawDataStrings[:,5]

array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], shape=(10000,), dtype='<U69')

In [63]:
np.array_equal(rawDataNumbers[:,0], np.array(rawDataStrings[:,5], dtype=np.int64))

True

In [66]:
rawDataStrings = np.delete(rawDataStrings,5,axis = 1)
rawDataStrings[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], shape=(10000,), dtype='<U69')

In [67]:
headerStrings = np.delete(headerStrings,5)
headerStrings

array(['issueDate', 'loan_status', 'termMonths', 'sub_grade', 'verification_status', 'addr_state'], dtype='<U19')

### State Address

In [68]:
headerStrings

array(['issueDate', 'loan_status', 'termMonths', 'sub_grade', 'verification_status', 'addr_state'], dtype='<U19')

In [69]:
headerStrings[5] = 'stateAdress'

In [71]:
np.unique(rawDataStrings[:,5])

array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [72]:
statesNames, statesCount = np.unique(rawDataStrings[:,5], return_counts=True)
statesCountSorted = np.argsort(-statesCount)
statesNames[statesCountSorted], statesCount[statesCountSorted]

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ', 'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'TN', 'CT', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY', 'KS', 'OK',
        'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DC', 'DE', 'WY', 'AK', 'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,  216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,   84,   83,
          74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,   25,   24,   17,   16,   10]))

In [74]:
rawDataStrings[:,5] = np.where(rawDataStrings[:,5] == '','0',rawDataStrings[:,5])

In [64]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [76]:
rawDataStrings[:,5] = np.where(np.isin(rawDataStrings[:,5], states_west),'1',rawDataStrings[:,5])
rawDataStrings[:,5] = np.where(np.isin(rawDataStrings[:,5], states_south),'2',rawDataStrings[:,5])
rawDataStrings[:,5] = np.where(np.isin(rawDataStrings[:,5], states_midwest),'3',rawDataStrings[:,5])
rawDataStrings[:,5] = np.where(np.isin(rawDataStrings[:,5], states_east),'4',rawDataStrings[:,5])

In [96]:
rawDataStrings = rawDataStrings.astype(np.float32)


In [97]:
rawDataStrings = rawDataStrings.astype(np.int16)

In [98]:
rawDataStrings

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]], shape=(10000, 6), dtype=int16)

## Converting to Numbers

In [ ]:
checkPointStrings = checkpoint('stringSectionDone', headerStrings,rawDataStrings)

NpzFile 'stringSectionDone.npz' with keys: header, data

### Checkpoint 1: Strings

## Manipulating Numeric Columns

### Substitute "Filler" Values

#### ID

#### Temporary Stats

#### Funded Amount

In [65]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill, 
                                  temporary_stats[0, columns_numeric[2]],
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

NameError: name 'loan_data_numeric' is not defined

#### Loaned Amount, Interest Rate, Total Payment, Installment

### Currency Change

#### The Exchange Rate

In [ ]:

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    


#### From USD to EUR

In [ ]:
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

#### Expanding the header

In [ ]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [ ]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

### Interest Rate

### Checkpoint 2: Numeric

In [ ]:
checkpoint_numeric['header'], checkpoint_numeric['data']

## Creating the "Complete" Dataset

## Sorting the New Dataset

## Storing the New Dataset